In [1]:
from datetime import datetime

from binance.client import Client
from binance.enums import *

from coinmarketcapapi import CoinMarketCapAPI, CoinMarketCapAPIError

from datetime import datetime

import numpy as np
import pandas as pd
import json
import time


In [2]:
client = Client('', '')

In [3]:
cmc = CoinMarketCapAPI('')

In [4]:
info = client.get_all_tickers()

In [5]:
quote_ccy = 'USDT'

In [6]:
ticker_list = [d['symbol'] for d in info if d['symbol'].endswith(quote_ccy)]

In [7]:
rank_info = cmc.cryptocurrency_map(sort='cmc_rank').data

In [8]:
target_rank_info = [d['symbol']+quote_ccy for d in rank_info if d['rank'] >= 300 and d['rank'] <= 350]

In [9]:
target_ticker_list = [d for d in ticker_list if d in target_rank_info]

In [10]:
target_ticker_list

['FETUSDT',
 'STPTUSDT',
 'ANTUSDT',
 'SUNUSDT',
 'FLMUSDT',
 'UTKUSDT',
 'XVSUSDT',
 'ALPHAUSDT',
 'CTKUSDT',
 'RIFUSDT',
 'ALICEUSDT',
 'PERPUSDT',
 'SUPERUSDT',
 'BAKEUSDT',
 'XVGUSDT',
 'GHSTUSDT',
 'TRIBEUSDT',
 'DARUSDT',
 'RGTUSDT',
 'MOVRUSDT']

In [11]:
target_ticker_list = ['BAKEUSDT']

In [12]:
def is_candidate(filtered_klines_info): 
    is_candidate = False
    
    light_context_kline_info = filtered_klines_info[:int(len(filtered_klines_info)/2)]
    
    strong_context_kline_info = filtered_klines_info[int(len(filtered_klines_info)/2): -1]
    
    buy_kline_info = filtered_klines_info[-1]
    
    light_context_kline_average_volume = np.mean(light_context_kline_info[:,5])
    strong_context_kline_average_volume = np.mean(strong_context_kline_info[:,5])
    buy_kline_average_volume = buy_kline_info[5]
    
    #Any Check to do with light context
    
    #Any check to do with Strong context
    
    for strong_context_kline in strong_context_kline_info:
        perc_change = strong_context_kline[-1]
        perc_volume = strong_context_kline[5] / buy_kline_average_volume
        
        if perc_change > 3 or perc_volume > 0.20:
            return False
    
    
    print('Passed Context Check')
    drop_from_high_gate = buy_kline_info[2]/buy_kline_info[4] < 1.06
    
    trigger_gate = 13 > buy_kline_info[-1] > 6
    print(buy_kline_info[-1])

    if trigger_gate and True:
        print('&&&&&&&&&&&&&&&& Passed Final Gate &&&&&&&&&&&&&&&&')
        is_candidate = True
    else:
        is_candidate = False

    return is_candidate

In [13]:
def find_percentage_increase_or_decrease(focused_kline_array):
    percentage_value = []
    for kline in focused_kline_array:
        diff = kline[4] - kline[1]
        perc_diff = diff/kline[1]
        percentage_value.append(perc_diff*100)
    return percentage_value 

In [14]:
trade_list = []
for target_ticker in target_ticker_list:
    
    print('Analyzing Hourly Started ')
    print(target_ticker)

    klines = client.get_historical_klines(target_ticker, Client.KLINE_INTERVAL_1HOUR, "1 day ago UTC") 
    length = int(len(klines))

    if(length < 24):
        continue
    
    array = np.array(klines, dtype=np.float64)
    focused_kline_array = np.delete(array, [6, 7, 8, 9, 10, 11], 1)
    percentage_value = find_percentage_increase_or_decrease(focused_kline_array)
    percentage_value = np.reshape(percentage_value, (length, 1))
    focused_kline_array = np.append(focused_kline_array, percentage_value, 1)

    for x in range(0, length):
        print(x)
        
        hit_ticker_dictionary = {}
        
        filtered_klines_info = focused_kline_array[x:x+24]
        
        print(filtered_klines_info)
        if(len(filtered_klines_info) == 24):
            print('Here')

            if(is_candidate(filtered_klines_info)):
                
                buy_kline_info = filtered_klines_info[-1]
                hit_ticker_dictionary = {}
                buy_price = buy_kline_info[4]
                hit_ticker_dictionary['ticker'] = target_ticker
                hit_ticker_dictionary['buy_price'] = buy_kline_info[4]

                buy_timesstamp = buy_kline_info[0]
                hit_ticker_dictionary['buy_datetime'] = str(datetime.fromtimestamp(buy_timesstamp // 1000))
                
                proft_percentage = 1.10
                loss_percentage = 0.75
                profit_sell_price = buy_price * proft_percentage
                loss_sell_price = buy_price * loss_percentage
                
                
                
                usdt = 1000
                
                buy_quantity = usdt/buy_price
                hit_ticker_dictionary['quantity'] = buy_quantity
                
                for y in range(x+25, length):
                    high_price = focused_kline_array[y-1:y][0][2]
                    low_price = focused_kline_array[y-1:y][0][3]
                    close_price = focused_kline_array[y-1:y][0][4]
                    sell_timesstamp = focused_kline_array[y-1:y][0][0]
                    sell_datetime = str(datetime.fromtimestamp(sell_timesstamp // 1000))


                    if high_price >= profit_sell_price:
                        hit_ticker_dictionary['sell_price'] = profit_sell_price

                        hit_ticker_dictionary['sell_datetime'] = str(datetime.fromtimestamp(sell_timesstamp // 1000))
                        difference = focused_kline_array[y-1:y][0][0] - buy_kline_info[0]
                        
                        hit_ticker_dictionary['days_to_gain_loss'] = round(difference/(1000 * 60 * 24 * 60))
                        price_difference = profit_sell_price - buy_price
                        hit_ticker_dictionary['price_diff'] = price_difference
                        hit_ticker_dictionary['proft_loss'] = usdt * proft_percentage - usdt
                        
                        break
                        
                    elif low_price <= loss_sell_price and False:
                        hit_ticker_dictionary['sell_price'] = loss_sell_price

                        hit_ticker_dictionary['sell_datetime'] = str(datetime.fromtimestamp(sell_timesstamp // 1000))
                        difference = focused_kline_array[y-1:y][0][0] - buy_kline_info[0]
                        
                        hit_ticker_dictionary['days_to_gain_loss'] = round(difference/(1000 * 60 * 24 * 60))
                        price_difference = loss_sell_price - buy_price
                        hit_ticker_dictionary['price_diff'] = price_difference
                        hit_ticker_dictionary['proft_loss'] = usdt * loss_percentage - usdt
                        break
                    elif sell_timesstamp - buy_timesstamp >= 21600000 :
                        hit_ticker_dictionary['sell_price'] = close_price

                        hit_ticker_dictionary['sell_datetime'] = str(datetime.fromtimestamp(sell_timesstamp // 1000))
                        difference = focused_kline_array[y-1:y][0][0] - buy_kline_info[0]
                        
                        hit_ticker_dictionary['days_to_gain_loss'] = round(difference/(1000 * 60 * 24 * 60))
                        price_difference = close_price - buy_price
                        hit_ticker_dictionary['price_diff'] = price_difference
                        hit_ticker_dictionary['proft_loss'] = buy_quantity * close_price - buy_quantity * buy_price
                        break
                        
                        
                        
                        
                        
                trade_list.append(hit_ticker_dictionary)
                
                busd = 100

                quantity = busd/ buy_kline_info[4]
                quantity_str = "{:0.0{}f}".format(quantity, 2)

                """
                order = client.create_test_order(symbol=target_ticker, side=SIDE_BUY, type='MARKET', quoteOrderQty=busd)
                time.sleep(5)
                order_id = order['orderId'] 
                '''order_id = client.get_all_orders(symbol=target_ticker)[0]['orderId'] '''

                buy_price = order['fills'][0]['price']
                buy_qty = order['executedQty']
                sell_price = buy_price * 1.03



                while client.get_order(symbol=target_ticker, orderId=order_id)['status'] != 'FILLED':
                    time.sleep(10)

                if client.get_order(symbol=target_ticker, orderId=order_id)['status'] == 'FILLED':
                    asset = target_ticker.split("USDT")[0]
                    balance = client.get_asset_balance(asset=asset)
                    free_to_trade = balance['free']
                    sell_price_str = "{:0.0{}f}".format(sell_price, 2)

                    order = client.create_test_order(symbol=target_ticker, side=SIDE_SELL, type='LIMIT', quantity=float(balance['free']), price=sell_price_str, timeInForce=TIME_IN_FORCE_GTC)
                    print('********** SELL ORDER SET ************************')

                    """


Analyzing Hourly Started 
BAKEUSDT
0
[[ 1.65808800e+12  3.02000000e-01  3.04000000e-01  3.00000000e-01
   3.03000000e-01  5.41743100e+05  3.31125828e-01]
 [ 1.65809160e+12  3.02000000e-01  3.06000000e-01  3.01000000e-01
   3.04000000e-01  2.84914300e+05  6.62251656e-01]
 [ 1.65809520e+12  3.05000000e-01  3.10000000e-01  3.04000000e-01
   3.09000000e-01  6.29001600e+05  1.31147541e+00]
 [ 1.65809880e+12  3.09000000e-01  3.13000000e-01  3.05000000e-01
   3.05000000e-01  1.18528070e+06 -1.29449838e+00]
 [ 1.65810240e+12  3.05000000e-01  3.10000000e-01  3.02000000e-01
   3.03000000e-01  1.40770660e+06 -6.55737705e-01]
 [ 1.65810600e+12  3.03000000e-01  3.16000000e-01  3.03000000e-01
   3.16000000e-01  1.13052140e+06  4.29042904e+00]
 [ 1.65810960e+12  3.16000000e-01  3.21000000e-01  3.14000000e-01
   3.18000000e-01  1.82269620e+06  6.32911392e-01]
 [ 1.65811320e+12  3.18000000e-01  3.23000000e-01  3.14000000e-01
   3.22000000e-01  1.24740260e+06  1.25786164e+00]
 [ 1.65811680e+12  3.220000

In [15]:
trade_list

[]

In [16]:
len(trade_list)

0

In [17]:
df = pd.DataFrame.from_dict(trade_list)

In [18]:
with pd.option_context('display.max_rows', None):
    display(df[df['price_diff'] > 0])

KeyError: 'price_diff'

In [ ]:
with pd.option_context('display.max_rows', None):
    display(df[df['price_diff'] < 0])

In [ ]:
with pd.option_context('display.max_rows', None):
    display(df[pd.isna(df['sell_price'])])

In [ ]:
df['proft_loss'].sum()

In [ ]:
with pd.option_context('display.max_rows', None):
    display(df[np.invert(pd.isna(df['sell_price']))])